Notebook for plotting fits and residuals

In [ ]:
from bayesbeat.result import get_fit
from bayesbeat.data import get_data, get_n_entries
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pathlib
from scipy import stats

from utils import (
    get_duration,
    get_frequency,
    compute_residuals,
)

plt.style.use("paper.mplstyle")

In [ ]:
outdir = pathlib.Path("figures")
outdir.mkdir(exist_ok=True)

file_format = "pdf"

Path to the data file

In [ ]:
data_file = "../data/PyTotalAnalysis_2024_02_23.mat"
n_ringdowns = get_n_entries(data_file)

In [ ]:
paths = {
    "simple_model_constant_noise": pathlib.Path("../analysis/gens_data/rundir/simple_model_constant_noise_fix/"),
    "simple_model_amp_noise": pathlib.Path("../analysis/gens_data/rundir/simple_model_both_noise_sources_fix/"),
    # "simple_model_amp_noise_a_scale": pathlib.Path("../analysis/rundir/simple_model_both_noise_sources_a_scale/"),
    "terms_1": pathlib.Path("../analysis/gens_data/rundir/analytic_model_1_term_both_noise_sources/"),
    "terms_3": pathlib.Path("../analysis/gens_data/rundir/analytic_model_3_terms_both_noise_sources/"),
    "terms_5": pathlib.Path("../analysis/gens_data/rundir/analytic_model_5_terms_both_noise_sources/"),
    # "terms_7": pathlib.Path("../analysis/gens_data/rundir/analytic_model_7_terms_both_noise_sources/"),
    "terms_7": pathlib.Path("../analysis/gens_data/rundir/analytic_model_7_terms_both_noise_sources_domega/"),
    # "terms_1_no_gap": pathlib.Path("../tmp_results/runs/outdir_compute_coeffs_constant_noise_1_terms_mcmc_no_gap_subset"),
    # "terms_3_no_gap": pathlib.Path("../tmp_results/runs/outdir_compute_coeffs_constant_noise_3_terms_mcmc_no_gap_subset"),
    # "terms_5_no_gap": pathlib.Path("../tmp_results/runs/outdir_compute_coeffs_constant_noise_5_terms_mcmc_no_gap_subset"),
    # "terms_7_no_gap": pathlib.Path("../tmp_results/runs/outdir_compute_coeffs_constant_noise_7_terms_mcmc_no_gap_subset"),
}

In [ ]:
index = 44
x_data, y_data, frequency, _ = get_data(data_file, index=index)

In [ ]:
results = {}
for key, path in paths.items():
    d = dict()
    result_file = path / "analysis" / f"index_{index}" / "result.hdf5"
    with h5py.File(result_file, "r") as res_file:
        posterior_samples = res_file["posterior_samples"][()]
        d["log_z"] = res_file["log_evidence"][()]
    # Get sample with max. log-likelihood
    max_logl_idx = np.argmax(posterior_samples["logL"])
    # Get noise sigmas
    
    d["sigma_constant"] = posterior_samples["sigma_constant_noise"][max_logl_idx]
    d["sigma_amp"] = (
        posterior_samples["sigma_amp_noise"][max_logl_idx]
        if "sigma_amp_noise" in posterior_samples.dtype.names else 0
    )
    d["y_fit"] = get_fit(
        config_file=next(path.glob("*.ini")),
        result_file=result_file,
        datafile=data_file,
        index=index,
        method="max",
        # device="cpu",
    )
    results[key] = d

In [ ]:
labels = {
    "simple_model_constant_noise": r"$M_1$, $\xi_2=0$",
    "simple_model_amp_noise": r"$M_1$, $\xi_2 \in [0, 1]$",
    "simple_model_amp_noise_a_scale": r"$M_1$, $\xi_2 \in [0, 1]$, $\alpha \neq 1$",
    "terms_1": r"$M_3$, $T=1$",
    "terms_3": r"$M_3$, $T=3$",
    "terms_5": r"$M_3$, $T=5$",
    "terms_7": r"$M_3$, $T=7$",
}

In [ ]:
for i, (key, res) in enumerate(results.items()):

    figsize = plt.rcParams["figure.figsize"].copy()
    figsize[0] = 1.8 * figsize[0]
    figsize[1] = 1.6 * figsize[1]
    fig, axs = plt.subplot_mosaic(
        [["fit", "fit", "fit", "empty"], ["fit", "fit", "fit", "empty"], ["res", "res", "res", "dist"]],
        figsize=figsize,
    )

    axs["fit"].scatter(x_data, y_data, color="k", s=1, lw=0.0, label="Data")

    axs["fit"].plot(x_data, res["y_fit"], c=f"C{i}", lw=0.5, label="Fit")
    axs["fit"].set_yscale("log")
    axs["fit"].legend()

    axs_zoom = axs["fit"].inset_axes(
        [0.05, 0.05, 0.35, 0.5],
        xlim=(320, 360),
        ylim=(1.5e-3, 4e-3),
    )
    axs_zoom.scatter(x_data, y_data, color="k", s=1, lw=0.0, label="Data")

    axs_zoom.plot(x_data, res["y_fit"], c=f"C{i}", lw=0.5, label="Fit")
    axs_zoom.set_yscale("log")
    axs_zoom.tick_params(axis="both", which="both", labelleft=False, labelbottom=False)
    axs["fit"].indicate_inset_zoom(axs_zoom, edgecolor="black")

    residuals = compute_residuals(
        y_data,
        res["y_fit"],
        res["sigma_constant"],
        res["sigma_amp"],
    )
    
    axs["res"].scatter(x_data, residuals, s=0.5, c=f"C{i}", lw=0.0)

    axs["dist"].hist(residuals, 32, histtype="step", orientation="horizontal", label=labels.get(key), color=f"C{i}", density=True)
    r = np.linspace(-5, 5, 1000)
    r_pdf = stats.norm.pdf(r)
    axs["dist"].plot(r_pdf, r, color="k", zorder=0)

    axs["empty"].axis("off")
    # axs["dist"].legend()
    # axs["fit"].grid()
    axs["res"].set_xlabel("Time [s]")
    axs["fit"].set_ylabel("Amplitude")
    # axs["res"].grid()
    axs["res"].set_ylabel(r"$\mathcal{R}$")
    axs["res"].sharex(axs["fit"])
    axs["res"].sharey(axs["dist"])
    axs["fit"].tick_params(labelbottom=False)
    axs["dist"].tick_params(labelleft=False)
    axs["dist"].set_xlabel("Counts")
    # axs["dist"].grid()
    # axs["res"].set_xscale("log")
    # axs["res"].set_xlim(5e2, 1e3)

    axs["empty"].text(0., 0.55, labels[key], fontsize=8)
    axs["empty"].text(0., 0.45, r"$\log_{10} Z = " + f"{res['log_z'] / np.log(10):.1f}" + r"$", fontsize=8)

    plt.tight_layout()
    fig.savefig(outdir / "fits" / f"{key}_fit_{index}.{file_format}")
    # break


In [ ]:
res = results["terms_5"]
residuals = compute_residuals(
        y_data,
        res["y_fit"],
        res["sigma_constant"],
        res["sigma_amp"],
    )

In [ ]:
noise = np.random.randn(*residuals.shape)
rfd = np.fft.rfft(residuals)
fbins = np.fft.rfftfreq(len(y_data), x_data[1] - x_data[0])
# fbins = np.fft.fftshift(fbins)

In [ ]:
fig, axs = plt.subplots()
axs.plot(fbins, np.abs(rfd) ** 2)

# axs.set_xlim(-0.1, 0.1)
# axs.set_xscale("log")
axs.set_xlabel("f [Hz]")
axs.set_ylabel("Power")

In [ ]:
1 / 2e-3